<a href="https://colab.research.google.com/github/dominikmeyer95/academic-output/blob/feature%2Fapplied_numerical_finance/applied_numerical_finance/estimating_greeks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applied Numerical Finance

* Group: Dominik Meyer, Mikhail Borovkov, Brage Bakken, Emanuele Chiarini, Enrico Giannelli
* Assignment: 10 Finite difference approach for the Greeks

# Key Resources & Notes

* Explain theoretical dimension of estimation methods in 1.1
* Estimate each Greek with all methods available and show potential difference in 1.2/1.3
* ...

# Table of Contents

* 1 [Option Price Sensitivities](#chapter_1)
    * 1.1 [Estimation Methods](#section_1_1)
        * 1.1.1 [Closed Form Solutions](#subsection_1_1_1)
        * 1.1.2 [Finite Difference Approximation](#subsection_1_1_2)
    * 1.2 [First Order Greeks](#section_1_2)
        * 1.2.1 [Delta: Option's Underlying Price Sensitivity](#subsection_1_2_1)
        * 1.2.2 [Vega: Option's Volatility Sensitivity](#subsection_1_2_2)
        * 1.2.3 [Theta: Option's Time Sensitivity](#subsection_1_2_3)
        * 1.2.4 [Rho: Option's Interest Rate Sensitivity](#subsection_1_2_4)
    * 1.3 [Second Order Greeks](#section_1_3)
        * 1.3.1 [Gamma: Delta's Underlying Price Sensitivity](#subsection_1_3_1)

<a class="anchor" name="chapter_1"></a>
# 1 Option Price Sensitivities

In [1]:
# import required packages
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# build function for option pricing
def price_option(S, K, r, t, sigma, direction, type, method):
    
    # calculate d1 and d2
    d1 = (np.log(S/K) + (r + sigma**2/2)*t)/(sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)
    
    # calculate european option prices via black_scholes
    if direction=='call' and type=='european' and method=='black_scholes':
        c = S*norm.cdf(d1, 0, 1) - K*np.exp(-r*t)*norm.cdf(d2, 0, 1)
        return c
    elif direction=='put' and type=='european' and method=='black_scholes':
        p = K*np.exp(-r*t)*norm.cdf(-d2, 0, 1) - S*norm.cdf(-d1, 0, 1)
        return p
    else:
        pass

In [3]:
#  build function for greek estimation
def estimate_greeks(S, K, r, t, sigma, direction, option_type, pricing_method, greek, estimation_method, h=0.01):
    """
    Estimate option greeks using closed form or finite differences.
    """

    # define general pricing parameters
    d1 = (np.log(S/K) + (r + sigma**2/2)*t)/(sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)

    # estimate delta
    if greek == "delta":
        # implement closed form estimation
        if estimation_method == "closed_form":
            if direction == "call":
                delta = norm.cdf(d1, 0, 1)
            elif direction == "put":
                delta = -norm.cdf(-d1, 0, 1)
            else:
                pass
        # implement finite difference estimation
        if estimation_method in ['backward_difference', 'forward_difference', 'central_difference']:
            # define finite difference parameters
            actual_price = price_option(S=S, K=K, r=r, t=t, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            forward_increment = price_option(S=S+h, K=K, r=r, t=t, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            backward_increment = price_option(S=S-h, K=K, r=r, t=t, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            # calculate finite differences
            if estimation_method == "backward_difference":
                delta = (actual_price - backward_increment)/h
            elif estimation_method == "forward_difference":
                delta = (forward_increment - actual_price)/h
            elif estimation_method == "central_difference":
                delta = (forward_increment - backward_increment)/(2*h)
                # if direction == "call" or direction == "put":
                # delta = finite_difference(lambda S: price_option(S=S, K=K, r=r, t=t, sigma=sigma, direction=direction, type=type, method=method), x=S, h=0.01, method="backward")
            else:
                pass
        else:
            pass
        return delta
    
    # estimate gamma
    elif greek == "gamma":
        # implement closed form estimation
        if estimation_method == "closed_form":
            if direction == "call" or direction == "put":
                gamma = norm.pdf(d1, 0, 1)/(S*sigma*np.sqrt(t))
            else:
                pass
        # implement finite difference estimation
        if estimation_method in ['backward_difference', 'forward_difference', 'central_difference']:
            # define finite difference parameters
            actual_price = price_option(S=S, K=K, r=r, t=t, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            forward_increment = price_option(S=S+h, K=K, r=r, t=t, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            double_forward_increment = price_option(S=S+2*h, K=K, r=r, t=t, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            backward_increment = price_option(S=S-h, K=K, r=r, t=t, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            double_backward_increment = price_option(S=S-2*h, K=K, r=r, t=t, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            # calculate finite differences
            if estimation_method == "backward_difference":
                gamma = (actual_price - 2*backward_increment + double_backward_increment)/(h**2)
            elif estimation_method == "forward_difference":
                gamma = (double_forward_increment - 2*forward_increment + actual_price)/(h**2)
            elif estimation_method == "central_difference":
                gamma = (forward_increment - 2*actual_price + backward_increment)/(h**2)
            else:
                pass
        return gamma
    
    # estimate vega
    elif greek == "vega":
        # implement closed form estimation
        if estimation_method == "closed_form":
            if direction == "call" or direction == "put":
                vega = S*norm.pdf(d1, 0, 1)*np.sqrt(t)
            else:
                pass
        # implement finite difference estimation
        if estimation_method in ['backward_difference', 'forward_difference', 'central_difference']:
            # define finite difference parameters
            actual_price = price_option(S=S, K=K, r=r, t=t, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            forward_increment = price_option(S=S, K=K, r=r, t=t, sigma=sigma+h, direction=direction, type=option_type, method=pricing_method)
            backward_increment = price_option(S=S, K=K, r=r, t=t, sigma=sigma-h, direction=direction, type=option_type, method=pricing_method)
            # calculate finite differences
            if estimation_method == "backward_difference":
                vega = (actual_price - backward_increment)/h
            elif estimation_method == "forward_difference":
                vega = (forward_increment - actual_price)/h
            elif estimation_method == "central_difference":
                vega = (forward_increment - backward_increment)/(2*h)
            else:
                pass
        # times by 0.01 for sensitivity to 1% change in volatility
        return vega*0.01
    
    # estimate theta
    elif greek == "theta":
        # implement closed form estimation
        if estimation_method == "closed_form":
            if direction == "call":
                theta = -(S*norm.pdf(d1, 0, 1)*sigma)/(2*np.sqrt(t)) - r*K*np.exp(-r*t)*norm.cdf(d2, 0, 1)
            elif direction == "put":
                theta = -(S*norm.pdf(d1, 0, 1)*sigma)/(2*np.sqrt(t)) + r*K*np.exp(-r*t)*norm.cdf(-d2, 0, 1)
            else:
                pass
        # implement finite difference estimation
        if estimation_method in ['backward_difference', 'forward_difference', 'central_difference']:
            # define finite difference parameters
            actual_price = price_option(S=S, K=K, r=r, t=t, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            forward_increment = price_option(S=S, K=K, r=r, t=t+h, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            backward_increment = price_option(S=S, K=K, r=r, t=t-h, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            # calculate finite differences
            if estimation_method == "backward_difference":
                theta = -(actual_price - backward_increment)/h
            elif estimation_method == "forward_difference":
                theta = -(forward_increment - actual_price)/h
            elif estimation_method == "central_difference":
                theta = -(forward_increment - backward_increment)/(2*h)
            else:
                pass
        # divide by 365 to get per day sensitivity
        return theta/365
    
    # estimate rho
    elif greek == "rho":
        # implement closed form estimation
        if estimation_method == "closed_form":
            if direction == "call":
                rho = K*t*np.exp(-r*t)*norm.cdf(d2, 0, 1)
            elif direction == "put":
                rho = -K*t*np.exp(-r*t)*norm.cdf(-d2, 0, 1)
            else:
                pass
        # implement finite difference estimation
        if estimation_method in ['backward_difference', 'forward_difference', 'central_difference']:
            # define finite difference parameters
            actual_price = price_option(S=S, K=K, r=r, t=t, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            forward_increment = price_option(S=S, K=K, r=r+h, t=t, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            backward_increment = price_option(S=S, K=K, r=r-h, t=t, sigma=sigma, direction=direction, type=option_type, method=pricing_method)
            # calculate finite differences
            if estimation_method == "backward_difference":
                rho = (actual_price - backward_increment)/h
            elif estimation_method == "forward_difference":
                rho = (forward_increment - actual_price)/h
            elif estimation_method == "central_difference":
                rho = (forward_increment - backward_increment)/(2*h)
            else:
                pass
        # times by 0.01 for sensitivity to 1% change in interest rate
        return rho*0.01

In [4]:
# quick sanity check (1/2)
greeks={}
greeks["delta"]=estimate_greeks(S=30, K=40, r=0.01, t=240/365, sigma=0.3, direction='call', option_type='european', pricing_method='black_scholes', greek='delta', estimation_method='closed_form')
greeks["gamma"]=estimate_greeks(S=30, K=40, r=0.01, t=240/365, sigma=0.3, direction='call', option_type='european', pricing_method='black_scholes', greek='gamma', estimation_method='closed_form')
greeks["vega"]=estimate_greeks(S=30, K=40, r=0.01, t=240/365, sigma=0.3, direction='call', option_type='european', pricing_method='black_scholes', greek='vega', estimation_method='closed_form')
greeks["theta"]=estimate_greeks(S=30, K=40, r=0.01, t=240/365, sigma=0.3, direction='call', option_type='european', pricing_method='black_scholes', greek='theta', estimation_method='closed_form')
greeks["roh"]=estimate_greeks(S=30, K=40, r=0.01, t=240/365, sigma=0.3, direction='call', option_type='european', pricing_method='black_scholes', greek='rho', estimation_method='closed_form')
greeks

{'delta': 0.15058613984880015,
 'gamma': 0.03203161102008452,
 'vega': 0.05686707929045143,
 'theta': -0.003663899299916886,
 'roh': 0.02632964262328151}

In [5]:
# quick sanity check (2/2)
greeks={}
greeks["delta"]=estimate_greeks(S=30, K=40, r=0.01, t=240/365, sigma=0.3, direction='call', option_type='european', pricing_method='black_scholes', greek='delta', estimation_method='central_difference')
greeks["gamma"]=estimate_greeks(S=30, K=40, r=0.01, t=240/365, sigma=0.3, direction='call', option_type='european', pricing_method='black_scholes', greek='gamma', estimation_method='central_difference')
greeks["vega"]=estimate_greeks(S=30, K=40, r=0.01, t=240/365, sigma=0.3, direction='call', option_type='european', pricing_method='black_scholes', greek='vega', estimation_method='central_difference')
greeks["theta"]=estimate_greeks(S=30, K=40, r=0.01, t=240/365, sigma=0.3, direction='call', option_type='european', pricing_method='black_scholes', greek='theta', estimation_method='central_difference')
greeks["roh"]=estimate_greeks(S=30, K=40, r=0.01, t=240/365, sigma=0.3, direction='call', option_type='european', pricing_method='black_scholes', greek='rho', estimation_method='central_difference')
greeks

{'delta': 0.15058619768681947,
 'gamma': 0.032031608232507836,
 'vega': 0.056843108939218956,
 'theta': -0.0036637773862466896,
 'roh': 0.02633427143720124}

In [6]:
# define common parameters
parameter = {
"otm stock price": 50,
"atm stock price": 100,
"itm stock price": 150,
"strike": 100,
"maturity": 1,
"rate": 0.00,
"sigma": 0.25,
"stock prices": np.arange(1, 250, 1),
"rates": np.arange(0.00, 1, 0.01),
"volatilities": np.arange(0.00, 1, 0.01),
"maturities": np.arange(0.01, 5, 0.01),
"increments": [3.0, 2.5, 2.0, 1.5, 1, 0.5, 0.1]
}

<a class="anchor" name="section_1_1"></a>
## 1.1 Estimation Methods

<a class="anchor" name="subsection_1_1_1"></a>
### 1.1.1 Closed Form Solutions

* show analytical representation

<a class="anchor" name="subsection_1_1_2"></a>
### 1.1.2 Finite Difference Approximation

* show numerical representation

<a class="anchor" name="section_1_2"></a>
## 1.2 First Order Greeks

<a class="anchor" name="subsection_1_2_1"></a>
### 1.2.1 Delta: Option's Underlying Price Sensitivity

In [7]:
# estimate greek
results = pd.DataFrame()
results['closed_form'] = estimate_greeks(S=parameter['stock prices'], K=parameter['strike'], r=parameter['rate'], t=parameter['maturity'], sigma=parameter['sigma'], direction='call', option_type='european', pricing_method='black_scholes', greek='delta', estimation_method='closed_form')
for h in parameter['increments']:
    finite_difference = estimate_greeks(S=parameter['stock prices'], K=parameter['strike'], r=parameter['rate'], t=parameter['maturity'], sigma=parameter['sigma'], direction='call', option_type='european', pricing_method='black_scholes', greek='delta', estimation_method='central_difference', h=h)
    estimation_error = results['closed_form'] - finite_difference
    results['error (h={})'.format(str(h))] = estimation_error

# plot greek
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_vline(x=parameter['strike'], line_width=3, line_dash="dash", line_color="yellow", name="strike price", secondary_y=False)
fig.add_scatter(x=parameter['stock prices'], y=results["closed_form"], mode='lines', name='closed form', secondary_y=False)
for h in parameter['increments']:
    fig.add_scatter(x=parameter['stock prices'], y=results["error (h={})".format(str(h))], mode='lines', name='error (h={})'.format(str(h)), secondary_y=True)
fig.update_layout(title="delta estimation", template="plotly_dark", title_x=0.5, showlegend=True)
fig.update_xaxes(title_text='stock prices', showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey")
fig.update_yaxes(title_text='delta level', showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=False)
fig.update_yaxes(title_text='error level', showgrid=False, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=True)
fig.show("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log
  """


<a class="anchor" name="subsection_1_2_2"></a>
### 1.2.2 Vega: Option's Volatility Sensitivity

In [8]:
# estimate greek
results = pd.DataFrame()
results['closed_form'] = estimate_greeks(S=parameter['atm stock price'], K=parameter['strike'], r=parameter['rate'], t=parameter['maturity'], sigma=parameter['volatilities'], direction='call', option_type='european', pricing_method='black_scholes', greek='vega', estimation_method='closed_form')
for h in parameter['increments']:
    finite_difference = estimate_greeks(S=parameter['atm stock price'], K=parameter['strike'], r=parameter['rate'], t=parameter['maturity'], sigma=parameter['volatilities'], direction='call', option_type='european', pricing_method='black_scholes', greek='vega', estimation_method='central_difference', h=h)
    estimation_error = results['closed_form'] - finite_difference
    results['error (h={})'.format(str(h))] = estimation_error

# plot greek
fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_vline(x=parameter['strike'], line_width=3, line_dash="dash", line_color="yellow", name="strike price", secondary_y=False)
fig.add_scatter(x=parameter['volatilities'], y=results["closed_form"], mode='lines', name='closed form', secondary_y=False)
for h in parameter['increments']:
    fig.add_scatter(x=parameter['volatilities'], y=results["error (h={})".format(str(h))], mode='lines', name='error (h={})'.format(str(h)), secondary_y=True)
fig.update_layout(title="vega estimation", template="plotly_dark", title_x=0.5, showlegend=True)
fig.update_xaxes(title_text='volatility', showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey")
fig.update_yaxes(title_text='vega level', showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=False)
fig.update_yaxes(title_text='error level', showgrid=False, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=True)
fig.show("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning:

invalid value encountered in true_divide



<a class="anchor" name="subsection_1_2_3"></a>
### 1.2.3 Theta: Option's Time Sensitivity

In [9]:
# estimate greek
results = pd.DataFrame()
results['closed_form'] = estimate_greeks(S=parameter['atm stock price'], K=parameter['strike'], r=parameter['rate'], t=parameter['maturities'], sigma=parameter['sigma'], direction='call', option_type='european', pricing_method='black_scholes', greek='theta', estimation_method='closed_form')
for h in parameter['increments']:
    finite_difference = estimate_greeks(S=parameter['atm stock price'], K=parameter['strike'], r=parameter['rate'], t=parameter['maturities'], sigma=parameter['sigma'], direction='call', option_type='european', pricing_method='black_scholes', greek='theta', estimation_method='central_difference', h=h)
    estimation_error = results['closed_form'] - finite_difference
    results['error (h={})'.format(str(h))] = estimation_error

# plot greek
fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_vline(x=parameter['strike'], line_width=3, line_dash="dash", line_color="yellow", name="strike price", secondary_y=False)
fig.add_scatter(x=parameter['maturities'], y=results["closed_form"], mode='lines', name='closed form', secondary_y=False)
for h in parameter['increments']:
    fig.add_scatter(x=parameter['maturities'], y=results["error (h={})".format(str(h))], mode='lines', name='error (h={})'.format(str(h)), secondary_y=True)
fig.update_layout(title="theta estimation", template="plotly_dark", title_x=0.5, showlegend=True)
fig.update_xaxes(title_text='maturity', showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey")
fig.update_yaxes(title_text='theta level', showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=False)
fig.update_yaxes(title_text='error level', showgrid=False, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=True)
fig.show("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning:

invalid value encountered in sqrt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning:

invalid value encountered in sqrt



<a class="anchor" name="subsection_1_2_4"></a>
### 1.2.4 Rho: Option's Interest Rate Sensitivity

In [10]:
# estimate greek
results = pd.DataFrame()
results['closed_form'] = estimate_greeks(S=parameter['atm stock price'], K=parameter['strike'], r=parameter['rates'], t=parameter['maturity'], sigma=parameter['sigma'], direction='call', option_type='european', pricing_method='black_scholes', greek='rho', estimation_method='closed_form')
for h in parameter['increments']:
    finite_difference = estimate_greeks(S=parameter['atm stock price'], K=parameter['strike'], r=parameter['rates'], t=parameter['maturity'], sigma=parameter['sigma'], direction='call', option_type='european', pricing_method='black_scholes', greek='rho', estimation_method='central_difference', h=h)
    estimation_error = results['closed_form'] - finite_difference
    results['error (h={})'.format(str(h))] = estimation_error

# plot greek
fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_vline(x=parameter['strike'], line_width=3, line_dash="dash", line_color="yellow", name="strike price", secondary_y=False)
fig.add_scatter(x=parameter['rates'], y=results["closed_form"], mode='lines', name='closed form', secondary_y=False)
for h in parameter['increments']:
    fig.add_scatter(x=parameter['rates'], y=results["error (h={})".format(str(h))], mode='lines', name='error (h={})'.format(str(h)), secondary_y=True)
fig.update_layout(title="rho estimation", template="plotly_dark", title_x=0.5, showlegend=True)
fig.update_xaxes(title_text='risk free rate', showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey")
fig.update_yaxes(title_text='rho level', showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=False)
fig.update_yaxes(title_text='error level', showgrid=False, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=True)
fig.show("")

<a class="anchor" name="section_1_3"></a>
## 1.3 Second Order Greeks

<a class="anchor" name="subsection_1_3_1"></a>
### 1.3.1 Gamma: Delta's Underlying Price Sensitivity

In [11]:
# estimate greek
results = pd.DataFrame()
results['closed_form'] = estimate_greeks(S=parameter['stock prices'], K=parameter['strike'], r=parameter['rate'], t=parameter['maturity'], sigma=parameter['sigma'], direction='call', option_type='european', pricing_method='black_scholes', greek='gamma', estimation_method='closed_form')
for h in parameter['increments']:
    finite_difference = estimate_greeks(S=parameter['stock prices'], K=parameter['strike'], r=parameter['rate'], t=parameter['maturity'], sigma=parameter['sigma'], direction='call', option_type='european', pricing_method='black_scholes', greek='gamma', estimation_method='central_difference', h=h)
    estimation_error = results['closed_form'] - finite_difference
    results['error (h={})'.format(str(h))] = estimation_error

# plot greek
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_vline(x=parameter['strike'], line_width=3, line_dash="dash", line_color="yellow", name="strike price", secondary_y=False)
fig.add_scatter(x=parameter['stock prices'], y=results["closed_form"], mode='lines', name='closed form', secondary_y=False)
for h in parameter['increments']:
    fig.add_scatter(x=parameter['stock prices'], y=results["error (h={})".format(str(h))], mode='lines', name='error (h={})'.format(str(h)), secondary_y=True)
fig.update_layout(title="gamma estimation", template="plotly_dark", title_x=0.5, showlegend=True)
fig.update_xaxes(title_text='stock prices', showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey")
fig.update_yaxes(title_text='gamma level', showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=False)
fig.update_yaxes(title_text='error level', showgrid=False, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=True)
fig.show("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning:

invalid value encountered in log

